# Hashing with SHAKE128 and SHAKE256

SHAKE128 and SHAKE256 algorithms differ from commonly used hashes (like MD5, SHA-1, SHA-2 or SHA3) by not having a fixed output size. 

Most hash algorithms have output size determined by their construction:

Algorithm      | Output Size (bits) | Output Size (bytes) | Hex Characters
-------------- | -----------------: | ------------------: | -------------:
MD5            | 128                | 16                   | 32
SHA-1          | 160                | 20                   | 40
**SHA-2**
SHA-224        | 224                | 28                   | 56
SHA-256        | 256                | 32                   | 64
SHA-384        | 384                | 48                   | 96
SHA-512        | 512                | 64                   | 128
SHA-512/224    | 224                | 28                   | 56
SHA-512/256    | 256                | 32                   | 64
**SHA-3** 
SHA3-224       | 224                | 28                   | 56
SHA3-256       | 256                | 32                   | 64
SHA3-384       | 384                | 48                   | 96
SHA3-512       | 512                | 64                   | 128

## Extendable Output Functions (XOFs)

With SHAKE algorithms (part of the SHA-3/Keccak family), you can take as much bits as you need. This is good for:

* Key derivation
* Checksums and integrity verifications
* Digital signatures, message authentication
* Pseudo random number generation
* Protocol padding

What is the difference between SHAKE128 and SHAKE256? The shorter variant is computationally easier, longer variant offers better security in long term.

## SHAKE in .NET

SHAKE algorithms are supported since .NET 8. However, due to their specific nature, they are _not_ derived from the common `HashAlgorithm` class and are used differently than other hash functions. They _do_ however support the static `HashData` and `HashDataAsync` methods for easy use:

In [3]:
using System.Security.Cryptography;

var sourceText = "Quick brown fox jumps over the lazy dog";
var sourceBytes = System.Text.Encoding.UTF8.GetBytes(sourceText);
var hashLengthsInBytes = new int[] { 5, 10, 16, 20, 32, 64 };

Console.WriteLine("SHAKE-128 Hashes:");
Console.WriteLine("Bits | Hash");
foreach (var length in hashLengthsInBytes){
    var hashBytes = Shake128.HashData(sourceBytes, length);
    Console.WriteLine($"{length * 8,4} | {Convert.ToHexString(hashBytes)}");
}
Console.WriteLine();

Console.WriteLine("SHAKE-256 Hashes:");
Console.WriteLine("Bits | Hash");
foreach (var length in hashLengthsInBytes){
    var hashBytes = Shake256.HashData(sourceBytes, length);
    Console.WriteLine($"{length * 8,4} | {Convert.ToHexString(hashBytes)}");
}


SHAKE-128 Hashes:
Bits | Hash
  40 | 68DD439EA3
  80 | 68DD439EA356CFF7B4BF
 128 | 68DD439EA356CFF7B4BFBC26C559091A
 160 | 68DD439EA356CFF7B4BFBC26C559091A97C7CE58
 256 | 68DD439EA356CFF7B4BFBC26C559091A97C7CE58A2FAAB2C975526F8095EFA00
 512 | 68DD439EA356CFF7B4BFBC26C559091A97C7CE58A2FAAB2C975526F8095EFA009EB5B8B6DC7C3FB170AA05477C6D060DCDDCDE421069968B0D6934192FFA034A

SHAKE-256 Hashes:
Bits | Hash
  40 | 339AB39E75
  80 | 339AB39E75A8A02F2A06
 128 | 339AB39E75A8A02F2A061E31D238EA96
 160 | 339AB39E75A8A02F2A061E31D238EA96348CB8C4
 256 | 339AB39E75A8A02F2A061E31D238EA96348CB8C46B2DFA149F8EF05412D45705
 512 | 339AB39E75A8A02F2A061E31D238EA96348CB8C46B2DFA149F8EF05412D45705EA41467D9332FD9633735E09E9F6EDE782644C0FA5157FACB95077A647CE5E2D


We can, however, read result as we like, without length specified upfront:

In [6]:
using (var xof = new Shake128()){
    xof.AppendData(sourceBytes);
    for (var i = 0; i < 64; i++){
        var hashByte = xof.Read(1);
        Console.Write($"{hashByte[0]:X2}");
        System.Threading.Thread.Sleep(100);
    }
}

68DD439EA356CFF7B4BFBC26C559091A97C7CE58A2FAAB2C975526F8095EFA009EB5B8B6DC7C3FB170AA05477C6D060DCDDCDE421069968B0D6934192FFA034A

## Benchmark

Let's compare speed of SHAKE128 and SHAKE256 to the fixed-length hashes:

In [10]:
using System.IO;
using System.Diagnostics;

static (string, TimeSpan) HashStream(Stream s, HashAlgorithm hash) {
    // Compute Hash
    Stopwatch sw = Stopwatch.StartNew();
    byte[] hashData = hash.ComputeHash(s);
    sw.Stop();

    // Convert bytes to hexadecimal string
    return (Convert.ToHexStringLower(hashData), sw.Elapsed);
}

static void HashFile(string label, string fileName, HashAlgorithm hash) {
    using var fs = File.OpenRead(fileName);
    var (hashString, time) = HashStream(fs, hash);
    Console.WriteLine($"{label,-12} | {time.TotalMilliseconds,4:F0} ms | {hashString}");
}

static void Shake128File(string fileName, int outputLength) {
    using var fs = File.OpenRead(fileName);

    // Compute hash
    var sw = Stopwatch.StartNew();
    byte[] hashData = Shake128.HashData(fs, outputLength / 8);
    sw.Stop();

    // Print result
    var hashString = Convert.ToHexStringLower(hashData);
    Console.WriteLine($"SHAKE128-{outputLength} | {sw.Elapsed.TotalMilliseconds,4:F0} ms | {hashString}");
}

static void Shake256File(string fileName, int outputLength) {
    using var fs = File.OpenRead(fileName);

    // Compute hash
    var sw = Stopwatch.StartNew();
    byte[] hashData = Shake256.HashData(fs, outputLength / 8);
    sw.Stop();

    // Print result
    var hashString = Convert.ToHexStringLower(hashData);
    Console.WriteLine($"SHAKE256-{outputLength} | {sw.Elapsed.TotalMilliseconds,4:F0} ms | {hashString}");
}

const string FileName = @"D:\Install\SQL_Server_2022_Express_Advanced.iso";
Console.WriteLine($"Hashing file {FileName} ({new FileInfo(FileName).Length / 1024 / 1024} MB):");
Console.WriteLine();
Console.WriteLine($"{"Hash",-12} | {"Time",-7} | Result");
Console.WriteLine(new string('-', 154));
HashFile("MD5", FileName, MD5.Create());
HashFile("SHA-1", FileName, SHA1.Create());
HashFile("SHA-256", FileName, SHA256.Create());
HashFile("SHA-384", FileName, SHA384.Create());
HashFile("SHA-512", FileName, SHA512.Create());
HashFile("SHA3-256", FileName, SHA3_256.Create());
HashFile("SHA3-384", FileName, SHA3_384.Create());
HashFile("SHA3-512", FileName, SHA3_512.Create());
Shake128File(FileName, 128);
Shake128File(FileName, 160);
Shake128File(FileName, 256);
Shake128File(FileName, 384);
Shake128File(FileName, 512);
Shake256File(FileName, 128);
Shake256File(FileName, 160);
Shake256File(FileName, 256);
Shake256File(FileName, 384);
Shake256File(FileName, 512);
Console.WriteLine(new string('-', 154));

Hashing file D:\Install\SQL_Server_2022_Express_Advanced.iso (623 MB):

Hash         | Time    | Result
----------------------------------------------------------------------------------------------------------------------------------------------------------
MD5          | 1421 ms | bcf84c500cd661e95bea8eebd7fbcd1b
SHA-1        | 1296 ms | 0570a2a468771efa8492cf3d74e8eefa5fc929c9
SHA-256      |  695 ms | e8c7cf1f4ad1a52895c99338d100d5a9c0ad6d51a6f76230e095d6f657683d65
SHA-384      | 1500 ms | c508cb2a75978c19dc826d5ad2c7e959078d603e2f511f8f9fdec8552b260f03c1492ec3baff663f011d92b7c632fb1a
SHA-512      | 1517 ms | 292431b2f3e8d97da2dfccac2e069609dbbc3e4fee43d928cb1ac903fab3b31c09567f6d0294aa677385637b71e5b40a6cfe42261a176d5a63fbd4900597d3d8
SHA3-256     | 2221 ms | d1bd951e53021f9dee5218566748faa940a96cf31a75ef139542bfb328baa601
SHA3-384     | 2832 ms | c0586208ca40ab228a066437033dd045be49ad794b27b70c482135aeb376410c4b40ceeb0321ba59bd0aaffd758fb431
SHA3-512     | 3780 ms | 8a5d02d8e61a70